# Data preparation

In [ ]:
# Separe aqui seus dados pra fazer a inferencia

# Setup Llama 3.1:8b Instruct

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os
import csv
from pathlib import Path
import json
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='llm_inferences.log',
    filemode='a'
)

In [ ]:
local_path = "/home/jovyan/datalake/models/llama3-8b"

print(f"Loading tokenizer from {local_path}...")
tokenizer = AutoTokenizer.from_pretrained(
    local_path,
    local_files_only=True,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("pad_token defined as eos_token for tokenizer.")

print(f"Loading model from {local_path}...")

model = AutoModelForCausalLM.from_pretrained(
    local_path,
    local_files_only=True,
    trust_remote_code=True,
    dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="cuda"
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [ ]:
generation_params = {
    "max_new_tokens": 256,
    "do_sample": False,
    #"temperature": 0.7,
    "truncation": False,
    "return_full_text": False,
}
batch_size = 64

In [ ]:
def save_batch_to_csv(batch, results_to_save):
    """
    Save a batch result into a CSV file, creating the header if the file
    does not exist or is empty.

    Args:
        batch (int): Batch ID
        results_to_save (list): List of llm results obtained (prompt + output)
    """

    output_csv_file = f"llm_inferences/llama3.1-8b-instruct/{batch}.csv"
    fieldnames = ["original_id", "original_prompt", "llm_response"]

    output_dir = Path(output_csv_file).parent
    output_dir.mkdir(parents=True, exist_ok=True)

    write_header = not os.path.exists(output_csv_file) or os.path.getsize(output_csv_file) == 0

    try:
        with open(output_csv_file, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            if write_header:
                writer.writeheader()
                logging.info(f"Header written to '{output_csv_file}'.")

            writer.writerows(results_to_save)

    except IOError as e:
        logging.error(f"Error on saving batch to CSV '{output_csv_file}': {e}")

In [ ]:
PROGRESS_FILE = 'inferences.json'

def load_progress():
    """Load the progress from a JSON file."""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            logging.info(f"Progress file '{PROGRESS_FILE}' found. Loading state.")
            return json.load(f)
    logging.info(f"No progress file '{PROGRESS_FILE}' found. Starting from zero.")
    return {}

def save_progress(progress_data):
    """Save the current progress into a JSON file"""
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress_data, f, indent=4)

In [ ]:
progress = load_progress()

In [ ]:
import pandas as pd
import logging
from tqdm import tqdm
import transformers
import os

# --- Configurações Iniciais ---
CHUNK_SIZE = 404
transformers.logging.set_verbosity_error()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 1. System Prompt Genérico e Direto
# A idea eh só declarar uma persona pro llama incorporar, bem como a tarefa a ser realizada
SYSTEM_PROMPT = (
    "Você é um classificador de texto altamente preciso de comentários do Reddit e Youtube. " #São os dois?
    "Sua única função é analisar a mensagem do usuário e responder estritamente com '0' ou '1'.\n"
    "1: O texto pertence à categoria solicitada.\n"
    "0: O texto NÃO pertence à categoria solicitada.\n"
    "Não forneça nenhuma explicação, apenas o número."
)

# 2. Exemplos de Few-Shot Estruturados como Diálogo
# Agr estamos meio que simulando uma conversa com exemplos pro modelo, ao inves de passar tudo como um contexto so
FEW_SHOT_CONFIG = {
    "toxicidade": {
        "examples": [
            {"role": "user", "content": "‘@ tá vendo que relato padrinho, que essas pilantras são capazes de aprontar, quem não ouvir seus conselhos vai pra sarjeta, enquanto a pilantras vai tá sentando em várias mandiocas’"},
            {"role": "assistant", "content": "1"},
            {"role": "user", "content": "‘Esse pai tem meu respeito. 👏🏻👏🏻👏🏻👏🏻’"},
            {"role": "assistant", "content": "0"},
            {"role": "user", "content": "‘Imagina aguentar essa doida berrando no teu ouvido todo santo dia.’"},
            {"role": "assistant", "content": "1"},
            {"role": "user", "content": "‘VERDADE CRIA JUIZO PAIZÃO, METE MARCHA’"},
            {"role": "assistant", "content": "0"},
            {"role": "user", "content": "‘meta mesmo gostoso mas não use drogas e não engravide usa camisinha e muito anticoncepcional meta mesmo vai tudo pro inferno seja feliz na terra‘"},
            {"role": "assistant", "content": "1"},
        ],
        "output_prefix": "results_toxic"
    },
    "misoginia": {
        "examples": [
            {"role": "user", "content": "‘@ tá vendo que relato padrinho, que essas pilantras são capazes de aprontar, quem não ouvir seus conselhos vai pra sarjeta, enquanto a pilantras vai tá sentando em várias mandiocas’"},
            {"role": "assistant", "content": "1"},
            {"role": "user", "content": "‘A gente precisa mesmo e de Deus’"},
            {"role": "assistant", "content": "0"},
            {"role": "user", "content": "‘Não precisa ser raparigueiro para cair nesse golpe, basta ter o azar de dar carona para uma vagabunda destas daí, aqui em Brasília muitos coroas caíram nesse golpe de dar carona para certas minas bonitas!’"},
            {"role": "assistant", "content": "1"},
            {"role": "user", "content": "‘valeu Copini, fiz varias  anotações para aplicar em um futuro próximo obrigado pelas dicas.’"},
            {"role": "assistant", "content": "0"},
            {"role": "user", "content": "‘cara, a mulherada de 17-18 aqui na escola tá tudo virada na pomba gira!😂 Elas tão distribuindo..vc nem chega elas que chegam...precisa correr atrás de Tia Neide não’"},
            {"role": "assistant", "content": "1"},
        ],
        "output_prefix": "results_mysogyny"
    }
}


for fold_number in range(1, 6):
    for task_name, config in FEW_SHOT_CONFIG.items():

        logging.info(f"============================================================")
        logging.info(f"INICIANDO PROCESSAMENTO: Fold {fold_number} - Tarefa: {task_name}")
        logging.info(f"============================================================")

        # Define o nome do arquivo de entrada dinamicamente
        sufixo_arquivo = ""
        if task_name == "toxicidade":
            sufixo_arquivo = "_TOXICIDADE"
        elif task_name == "misoginia":
            sufixo_arquivo = "_MISOGINIA"

        input_filename = f"comentarios_fold_{fold_number}_estratificado{sufixo_arquivo}.csv"

        if not os.path.exists(input_filename):
            logging.warning(f"Arquivo {input_filename} não encontrado. Pulando para a próxima tarefa.")
            continue

        all_results_for_run = []
        reader = pd.read_csv(input_filename, chunksize=CHUNK_SIZE)

        for chunk_index, chunk_df in enumerate(tqdm(reader, desc=f"Fold {fold_number} ({task_name})")):
            if chunk_df.empty:
                continue

            batch_messages_text = chunk_df['comentario'].astype(str).tolist()
            batch_conversations = []

            # 3. Construção Dinâmica da Conversa para Cada Mensagem
            for msg_text in batch_messages_text:
                # Inicia a conversa com o prompt de sistema
                conversation = [{"role": "system", "content": SYSTEM_PROMPT}]
                
                # Adiciona os exemplos de few-shot específicos da tarefa
                conversation.extend(config["examples"])
                
                # Adiciona a mensagem real a ser classificada no final
                conversation.append({"role": "user", "content": msg_text})
                
                batch_conversations.append(conversation)

            if not batch_conversations:
                continue

            # O resto do código funciona como antes, mas agora com o prompt conversacional
            batch_prompts_formatted = tokenizer.apply_chat_template(
                batch_conversations,
                tokenize=False,
                add_generation_prompt=True
            )

            batch_results = pipe(batch_prompts_formatted, **generation_params)

            results_to_save_this_chunk = []
            for i, result in enumerate(batch_results):
                llm_response = result[0]['generated_text'].strip()
                original_message = batch_messages_text[i]

                result_to_save = {
                    "original_id": chunk_df['id_comentario_anonimizado'].iloc[i],
                    # Salva apenas a mensagem original, pois o prompt completo é grande e repetitivo
                    "original_prompt": original_message,
                    "llm_response": llm_response
                }
                results_to_save_this_chunk.append(result_to_save)

            if results_to_save_this_chunk:
                all_results_for_run.extend(results_to_save_this_chunk)

        # Após processar todos os chunks, salva o resultado consolidado
        if all_results_for_run:
            output_prefix = config["output_prefix"]
            output_filename = f"{output_prefix}_{fold_number}.csv"

            final_df = pd.DataFrame(all_results_for_run)
            final_df.to_csv(output_filename, index=False)
            logging.info(f"Resultados consolidados salvos em: {output_filename}")
        else:
            logging.warning(f"Nenhum resultado gerado para Fold {fold_number} - Tarefa: {task_name}")

logging.info("============================================================")
logging.info("TODAS AS TAREFAS FORAM CONCLUÍDAS!")
logging.info("============================================================")

Fold 1 (toxicidade): 1it [01:57, 117.08s/it]
Fold 1 (misoginia): 1it [01:27, 87.60s/it]
Fold 2 (toxicidade): 1it [02:01, 121.69s/it]
Fold 2 (misoginia): 1it [01:27, 88.00s/it]
Fold 3 (toxicidade): 1it [02:02, 122.91s/it]
Fold 3 (misoginia): 1it [01:28, 88.31s/it]
Fold 4 (toxicidade): 1it [02:00, 120.56s/it]
Fold 4 (misoginia): 1it [01:27, 87.77s/it]
Fold 5 (toxicidade): 1it [02:00, 120.84s/it]
Fold 5 (misoginia): 1it [01:28, 88.31s/it]
